In [85]:
import os
import sys
import csv
import requests
import urllib.request
import json
import pandas as pd
import psycopg2
from sqlalchemy import create_engine
from textblob import TextBlob
import nltk

sys.path.append('../')
from envir import config
import imp
imp.reload(config)

<module 'envir.config' from '../envir/config.py'>

In [86]:
con = psycopg2.connect(config.con)

In [94]:
for_col = pd.read_sql('''SELECT * FROM clean.data_for_model_share LIMIT 10''', con)

In [96]:
list(for_col.columns)

['minor',
 'crucial',
 'significant',
 'business_id',
 'city',
 'inspection_date',
 'name',
 'establishment_type',
 'address',
 'latitude',
 'longitude',
 'inspection_result',
 'violations_desc',
 'review_count',
 'attributes',
 'relative_date_6_months',
 'sentiment',
 'stars',
 'Acai Bowls',
 'Active Life',
 'Adult Entertainment',
 'Afghan',
 'African',
 'Airport Lounges',
 'Airport Terminals',
 'Airports',
 'American (New)',
 'American (Traditional)',
 'Amusement Parks',
 'Antiques',
 'Arabian',
 'Arcades',
 'Argentine',
 'Armenian',
 'Art Classes',
 'Art Galleries',
 'Arts & Crafts',
 'Arts & Entertainment',
 'Asian Fusion',
 'Auto Repair',
 'Automotive',
 'Bagels',
 'Bakeries',
 'Bangladeshi',
 'Barbeque',
 'Barbers',
 'Bars',
 'Basque',
 'Beach Bars',
 'Beauty & Spas',
 'Bed & Breakfast',
 'Beer',
 'Beer Bar',
 'Beer Garden',
 'Beer Gardens',
 'Belgian',
 'Bistros',
 'Books',
 'Bookstores',
 'Bowling',
 'Brasseries',
 'Brazilian',
 'Breakfast & Brunch',
 'Breweries',
 'Brewpubs',


In [65]:
pd.read_sql('''SELECT count(*) FROM clean.review_keywords_chunked_2 LIMIT 10''', con)

,count
0,939233


In [78]:
pd.read_sql('''SELECT * FROM clean.review_keywords_chunked_2 LIMIT 10''', con)

,review_id,business_id,date,count: i_love,count: old_school,count: pushy,count: pool,count: affordable,count: christ,count: stench,...,count: terrible,count: horrible,count: impressed,count: grody,count: nasty,count: hair,count: rat,count: hands,count: gross,count: wash_their_hands
0,MTYNpnH2C9ZhSEtwQ90_ZQ,01l8MH9tBK6GPvJQdMU1gw,2017-05-28 23:58:48,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
1,-StH6VejSJTMXVTa4eysAg,01l8MH9tBK6GPvJQdMU1gw,2016-02-15 06:25:43,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
2,01hnTvyvHjte4sGvuKm2pg,01l8MH9tBK6GPvJQdMU1gw,2017-11-07 20:38:39,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,NoNeYYzRAxik8xSb4yVW4Q,01l8MH9tBK6GPvJQdMU1gw,2014-04-15 02:38:05,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,jNo4s-_eueBx6zEXdwiKzA,01l8MH9tBK6GPvJQdMU1gw,2015-11-18 15:18:36,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
5,TNpm4Hs6x2yJvYbUZSeA4w,01l8MH9tBK6GPvJQdMU1gw,2015-09-07 02:40:30,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,jCW69qfzeL89pfLLPY81IQ,01l8MH9tBK6GPvJQdMU1gw,2017-02-16 03:49:32,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,9xrmSsFwiiArSZez550bwg,01l8MH9tBK6GPvJQdMU1gw,2014-03-14 15:24:09,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,ywtCoKYuzKEPyuMt5DucSA,01l8MH9tBK6GPvJQdMU1gw,2016-03-14 03:07:02,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,OriPsMQx1cRyE9hgNnhQFg,01l8MH9tBK6GPvJQdMU1gw,2016-11-26 04:29:09,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [87]:
tables = pd.read_sql("select * from pg_tables WHERE schemaname = 'clean'", con)

In [88]:
con = psycopg2.connect(config.con)
cur = con.cursor()
for id in ['yg833', 'jl9489']:
    for table in tables[tables['tableowner']=='sjf374']['tablename']:
        cur.execute('''GRANT ALL PRIVILEGES on Table clean.{table} to {id};'''.format(id=id, table=table))                                      
con.commit()

In [89]:
tables[tables['tableowner']=='sjf374']['tablename']

1                 nyc_inspections
5       review_keywords_chunked_2
6                review_sentiment
7                 review_keywords
8             review_keywords_new
9         review_keywords_chunked
10    business_dummied_categories
11                 data_for_model
12           data_for_model_share
Name: tablename, dtype: object

In [90]:
tables

,schemaname,tablename,tableowner,tablespace,hasindexes,hasrules,hastriggers
0,clean,toronto_inspections_cleaned,jl9489,None,False,False,False
1,clean,nyc_inspections,sjf374,None,False,False,False
2,clean,violations_matched_yelp_id,yg833,None,True,False,False
3,clean,clean_vegas_violations,yg833,None,True,False,False
4,clean,violations_matched_yelp_id2,yg833,None,True,False,False
5,clean,review_keywords_chunked_2,sjf374,None,False,False,False
6,clean,review_sentiment,sjf374,None,True,False,False
7,clean,review_keywords,sjf374,None,True,False,False
8,clean,review_keywords_new,sjf374,None,True,False,False
9,clean,review_keywords_chunked,sjf374,None,False,False,False


In [55]:
pd.read_sql('''SELECT * FROM pg_stat_activity
WHERE datname !='rdsadmin'
and usename = 'sjf374';''', con)

,datid,datname,pid,usesysid,usename,application_name,client_addr,client_hostname,client_port,backend_start,xact_start,query_start,state_change,waiting,state,backend_xid,backend_xmin,query
0,1764161,yelp_abcs,22717,1764017,sjf374,,None,None,-1,2019-12-14 17:37:25.373638+00:00,2019-12-14 17:37:25.386044+00:00,2019-12-14 17:37:25.386113+00:00,2019-12-14 17:37:25.386139+00:00,False,idle in transaction,None,None,show standard_conforming_strings
1,1764161,yelp_abcs,22716,1764017,sjf374,,None,None,-1,2019-12-14 17:37:24.468153+00:00,2019-12-14 17:37:58.116559+00:00,2019-12-14 23:56:47.967081+00:00,2019-12-14 23:56:47.967795+00:00,False,idle in transaction,None,None,SELECT * FROM clean.review_keywords LIMIT 10
2,1764161,yelp_abcs,32206,1764017,sjf374,,None,None,-1,2019-12-14 19:23:25.840144+00:00,2019-12-14 19:24:00.187896+00:00,2019-12-14 21:32:27.565921+00:00,2019-12-14 21:32:32.691761+00:00,False,idle in transaction,None,None,SELECT * FROM clean.review_sentiment
3,1764161,yelp_abcs,47917,1764017,sjf374,,None,None,-1,2019-12-14 22:05:34.700421+00:00,NaT,2019-12-15 15:22:06.961135+00:00,2019-12-15 15:22:06.961301+00:00,False,idle,None,None,COMMIT
4,1764161,yelp_abcs,24972,1764017,sjf374,,None,None,-1,2019-12-15 15:05:08.145068+00:00,2019-12-15 15:23:15.488590+00:00,2019-12-15 15:23:15.488775+00:00,2019-12-15 15:23:15.488776+00:00,False,active,None,42869365,SELECT * FROM pg_stat_activity\nWHERE datname ...


In [40]:
pd.read_sql('''SELECT pg_cancel_backend(24623);''', con)



,pg_cancel_backend
0,False


In [41]:
pd.read_sql('''SELECT pg_cancel_backend(22716);''', con)

,pg_cancel_backend
0,True


In [42]:
pd.read_sql('''SELECT pg_cancel_backend(32206);''', con)

,pg_cancel_backend
0,True


In [43]:
pd.read_sql('''SELECT pg_cancel_backend(24829);''', con)

,pg_cancel_backend
0,False


In [44]:
pd.read_sql('''SELECT pg_cancel_backend(22717);''', con)

,pg_cancel_backend
0,True


In [45]:
pd.read_sql('''SELECT pg_cancel_backend(47917);''', con)

,pg_cancel_backend
0,True


In [48]:
pd.read_sql('SELECT * FROM clean.review_sentiment limit 10', con)['date']

0   2014-04-07 23:04:06
1   2016-03-06 23:56:52
2   2018-02-21 17:29:04
3   2017-03-29 02:09:18
4   2014-09-21 12:37:38
5   2014-09-09 22:16:11
6   2016-07-14 03:29:57
7   2016-08-15 22:53:36
8   2018-03-13 16:37:22
9   2015-07-16 17:12:01
Name: date, dtype: datetime64[ns]